In [ ]:
!jupyter notebook --version

### Additional Uses

#### Pattern: Open-Close

- Open File
- Operate on open file
- Close File

or...

- Open socket
- Operate on socket
- Close socket

#### Pattern: Start-Stop

- Start database transaction
- Perform database operations
- Commit or rollback transaction

or

- Start timer
- perform operations
- Stop timer

#### Pattern: Lock-Release

- acquire thread lock
- perform some operations
- release thread lock

#### Pattern: Change-Reset

- change Decimal context precision
- perform some operations using the new precision
- reset Decimal context precision back to original value

or

- redirect stdout to a file
- perform some operations that write to stdout
- reset stdout to original value

#### Pattern: Wacky Stuff!

In [ ]:
with tag('p'):
    print('some text', end='')

View the below code for this printout

Or you can nest them like this:

In [ ]:
with tag('p'):
    print('some', end='')
    with tag('b'):
        print('bold ', end='')
    print('text', end='')

Again, review the below code for the result of this

Listed below are more examples....

#### Code Examples

##### Decimals * Change-Reset Pattern

In [ ]:
import decimal

In [ ]:
decimal.getcontext()

In [ ]:
decimal.getcontext().prec = 14

In [ ]:
decimal.getcontext()

In the above stdout, the prec *should* be 14 not 28...

This is because of a bug in Jupyter, as noted by Fred.

In [ ]:
old_prec = decimal.getcontext().prec
decimal.getcontext().prec = 4
print(decimal.Decimal(1) / decimal.Decimal(3))
decimal.getcontext().prec = old_prec
print(decimal.Decimal(1) / decimal.Decimal(3))

In [ ]:
class precision:
    def __init__(self, prec):
        self.prec = prec
        self.current_prec = decimal.getcontext().prec
        
    def __enter__(self):
        decimal.getcontext().prec = self.prec
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        decimal.getcontext().prec = self.current_prec
        return False

In [ ]:
with precision(3):
    print(decimal.Decimal(1) / decimal.Decimal(3))
print(decimal.Decimal(1) / decimal.Decimal(3))

The decimal module has a built-in context manager too!

In [ ]:
with decimal.localcontext() as ctx:
    ctx.prec = 3
    print(decimal.Decimal(1) / decimal.Decimal(3))
print(decimal.Decimal(1) / decimal.Decimal(3))  

##### Start-Stop Pattern

In [ ]:
from time import perf_counter, sleep

In [ ]:
class Timer:
    def __init__(self):
        self.elapsed = 0
        
    def __enter__(self):
        self.start = perf_counter()
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start
        return False

In [ ]:
with Timer() as timer:
    sleep(1)
    
print(timer.elapsed)

##### Redirect stdout

In [ ]:
print('hello')

In [ ]:
import sys

class OutToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stdout = sys.stdout
        
    def __enter__(self):
        self._file = open(self._fname, 'w')
        sys.stdout = self._file
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stdout = self._current_stdout
        self._file.close()
        return False

In [ ]:
with OutToFile('test2.txt'):
    print('Writing the stdout to file instead of console!')

Notice there is no output! (It was written to file instead of console...but now if you print...)

In [ ]:
print('hello')

In [ ]:
sys.stdout

##### Injecting opening and closing HTML tags

In [ ]:
class Tag:
    def __init__(self, tag):
        self._tag = tag
        
    def __enter__(self):
        print(f'<{self._tag}>', end='')
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        print(f'</{self._tag}>', end='')
        return False

In [ ]:
with Tag('p'):
    print('some ', end='')
    with Tag('b'):
        print('bold', end='')
    print(' text', end='')

##### Re-Entrant Context Manager

This will call the \_\_enter\_\_ method multiple times (as well as the \_\_exit\_\_ method multiple times to close off)

In [ ]:
'''
Title
- Items 1
    - sub item 1a
    - sub item 1b
-Items 2
    - sub item 2a
    - sub item 2b
'''

In [38]:
class ListMaker:
    def __init__(self, title, prefix='- ', indent=3):
        self._title = title
        self._prefix = prefix
        self._indent = indent
        self._current_indent = 0
        print(self._title)

In [39]:
lm = ListMaker('Items')

Items


In [44]:
class ListMaker:
    def __init__(self, title, prefix='- ', indent=3):
        self._title = title
        self._prefix = prefix
        self._indent = indent
        self._current_indent = 0
        print(self._title)
        
    def __enter__(self):
        self._current_indent += self._indent
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        self._current_indent -= self._indent
        return False
    
    def print(self, arg):
        s = ' ' * self._current_indent + self._prefix + str(arg)
        print(s)

In [45]:
with ListMaker('Items') as lm:
    lm.print('Item 1')
    lm.print('Item 2')

Items
   - Item 1
   - Item 2


In [48]:
with ListMaker('Items') as lm:
    lm.print('Item 1')
    with lm:
        lm.print('sub item 1a')
        lm.print('sub item 1b')
        with lm:
            lm.print('yeet')
    lm.print('Item 2')
    with lm:
        lm.print('sub item 2a')
        lm.print('sub item 2b')
        lm.print('sub item 2c')
        lm.print('sub item 2d')        

Items
   - Item 1
      - sub item 1a
      - sub item 1b
         - yeet
   - Item 2
      - sub item 2a
      - sub item 2b
      - sub item 2c
      - sub item 2d


Notice how the nesting works!

Now ofc, this can be re-directed!

In [52]:
with OutToFile('my_list.txt'):
    with ListMaker('Items') as lm:
        lm.print('Item 1')
        with lm:
            lm.print('sub item 1a')
            lm.print('sub item 1b')
            with lm:
                lm.print('yeet')
        lm.print('Item 2')
        with lm:
            lm.print('sub item 2a')
            lm.print('sub item 2b')
            lm.print('sub item 2c')
            lm.print('sub item 2d') 

And the above code printed that same stdout, but instead of to console it went to a file!